In [49]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [54]:
iris_data = load_iris()
iris_df = pd.DataFrame(data = iris_data['data'], columns = iris_data['feature_names'])
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [55]:
FEATURES = ['sepal length (cm)', 'petal length (cm)', 'petal width (cm)']
TARGET = 'sepal width (cm)'

X = iris_df[FEATURES]
y = iris_df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 3), (38, 3), (112,), (38,))

# Custom dataset

In [56]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, x, y=None):
    self.x = torch.tensor(x.values, dtype=torch.float32).to(device)
    self.y = torch.tensor(y.values, dtype=torch.float32).to(device) if y is not None else None

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx] if self.y is not None else torch.empty((1, 1), dtype=torch.float32)

In [57]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test)

In [58]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Create Network

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PM2_5(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(3, 128)
    self.fc2 = nn.Linear(128, 128)
    self.output = nn.Linear(128, 1)

  def forward(self, x):
    x = self.flatten(x)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    logits = self.output(x)
    return logits

In [61]:
learning_rate = 1e-3
epochs = 1000

In [62]:
model = PM2_5().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model

In [63]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  loss_history = []
  score_history = []

  for batch, (X, y) in enumerate(dataloader):
    # compute prediction and loss
    pred = model(X).squeeze(1) # forward pass
    loss = loss_fn(pred, y)

    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 2.0)
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      loss_history.append(loss)

      score = mean_squared_error(y.cpu().detach().numpy().tolist(), pred.cpu().detach().numpy().tolist(), squared=True)
      score_history.append(score)

      print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')

  return loss_history, score_history


def predict(dataloader, model):
  final_preds = []

  model.eval()
  with torch.no_grad():
    for X, _ in dataloader:
      pred = model(X).squeeze(1)
      final_preds.extend(pred.cpu().detach().numpy().tolist())

  return final_preds

In [64]:
loss_history = []
score_history = []

for epoch in range(epochs):
   print(f"Epoch {epoch + 1}\n-------------------------------")
   epoch_loss_history, epoch_score_history = train_loop(train_dataloader, model, loss_fn, optimizer)
   loss_history.extend(epoch_loss_history)
   score_history.extend(epoch_score_history)

Epoch 1
-------------------------------
loss: 6.400157  [    0/  112]
Epoch 2
-------------------------------
loss: 0.309467  [    0/  112]
Epoch 3
-------------------------------
loss: 0.003943  [    0/  112]
Epoch 4
-------------------------------
loss: 0.141572  [    0/  112]
Epoch 5
-------------------------------
loss: 0.109189  [    0/  112]
Epoch 6
-------------------------------
loss: 0.183030  [    0/  112]
Epoch 7
-------------------------------
loss: 0.134112  [    0/  112]
Epoch 8
-------------------------------
loss: 0.037415  [    0/  112]
Epoch 9
-------------------------------
loss: 0.050300  [    0/  112]
Epoch 10
-------------------------------
loss: 0.332855  [    0/  112]
Epoch 11
-------------------------------
loss: 0.208598  [    0/  112]
Epoch 12
-------------------------------
loss: 0.016532  [    0/  112]
Epoch 13
-------------------------------
loss: 0.131131  [    0/  112]
Epoch 14
-------------------------------
loss: 0.088475  [    0/  112]
Epoch 15
------

In [65]:
y_pred = predict(test_dataloader, model)

In [66]:
mean_squared_error(y_test, y_pred)

0.06301689156820096